In [1]:
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import LeaveOneOut,StratifiedKFold, cross_val_score
from sklearn.feature_selection import f_classif, SelectKBest, VarianceThreshold, RFECV, SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.learning_curve import learning_curve


In [ ]:
def model_generation(file_name_1,file_name_2,output_file_dir):
    x = [] 
    y = []
    with open(file_name_1) as f:
        feature_name = f.readline()
        feature_name = feature_name.rstrip(os.linesep).split(',')
        feature_name = feature_name[1:]
        for line in f:
            x.append([])
            temp = line.rstrip(os.linesep).split(',')
            temp = temp[1:]
            for i in temp:
                if i == '':
                    x[-1].append(np.nan)
                else:
                    x[-1].append(float(i))
        f.close()
    with open(file_name_2) as f:
        for line in f:
            temp = line.rstrip(os.linesep)
            y.append(float(temp))
        f.close()
    x = np.array(x)
    x = x.astype('float32')
    y2 = []
    for i in y:
        if i >= 0.3:
            y2.append(0)
        elif i>= -1:
            y2.append(1)
        else:
            y2.append(2)
    isnan = ~np.isnan(x).any(axis=0)
    x = x[:,isnan]
    feature_name = [feature_name[i] for i in range(len(isnan)) if isnan[i] ]
    isinf = ~np.isinf(x).any(axis=0)
    x = x[:,isinf]
    feature_name = [feature_name[i] for i in range(len(isinf)) if isinf[i] ]
    sel = VarianceThreshold()
    x = sel.fit_transform(x)
    feature_name = [feature_name[i] for i in range(len(sel.get_support())) if sel.get_support()[i]]
    isneginf = ~np.isneginf(x).any(axis=0)
    x = x[:,isneginf]
    feature_name = [feature_name[i] for i in range(len(isneginf)) if isneginf[i] ]
    clf = ExtraTreesClassifier(100000)
    clf = clf.fit(x, y2)
    importances = clf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in clf.estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")
    for feature in range(x.shape[1]):
        print("%d. %s (%f)" % (feature + 1, feature_name[indices[feature]], importances[indices[feature]]))
    plt.figure()
    plt.plot(importances[indices])
    plt.title('Feature Importance in Descending Order')
    plt.xlabel('Ranking')
    plt.ylabel('Importance score')
    plt.savefig(output_file_dir+'feature_importance_des_order.png')
    plt.show()
    plt.figure()
    sns.distplot(importances[indices])
    plt.title('Feature Importance Distribution')
    plt.xlabel('Count')
    plt.ylabel('Importance score')
    plt.savefig(output_file_dir+'feature_importance_distribution.png')
    plt.show()
    threshold = np.mean(importances[indices])+2*np.std(importances[indices])
    for i in range(len(importances)):
        if importances[indices[i]]<threshold:
            cutoff_feature_index = i
            break
    print('Number of features left:'+str(cutoff_feature_index))
    x_selected = x[:,indices[0:cutoff_feature_index]]
    feature_name = feature_name[indices[0:cutoff_feature_index]]

    lda = LinearDiscriminantAnalysis()
    lr = LogisticRegression()
    svc = SVC(kernel="linear")

    print('LDA:')
    lda.fit(x_selected,y2)
    y_pred = lda.predict(x_selected)
    print(classification_report(y2, y_pred))

    print('Logistic regression:')
    lr.fit(x_selected,y2)
    y_pred = lr.predict(x_selected)
    print(classification_report(y2, y_pred))

    print('Support vector classification with linear kernel:')
    svc.fit(x_selected,y2)
    y_pred = svc.predict(x_selected)
    print(classification_report(y2, y_pred))
    
    df = pd.DataFrame(x_selected,columns=[feature_name[indices[i]] for i in range(cutoff_feature_index)])
    corr = df.corr()
    microarray_cmap = LinearSegmentedColormap('microarray', {
            'red': [(0.0, 1.0, 1.0), (0.5, 0.2, 0.2), (1.0, 0.0, 0.0)],
            'green': [(0.0, 0.0, 0.0), (0.5, 0.2, 0.2), (1.0, 1.0, 1.0)],
            'blue': [(0.0, 0.0, 0.0), (0.5, 0.2, 0.2), (1.0, 0.0, 0.0)],
    })
    microarray_cmap.set_bad('w')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    mask =  np.tri(corr.shape[0],k=-1).T
    cax = ax.matshow(np.ma.array(corr,mask=mask), interpolation='none', cmap=microarray_cmap)
    fig.colorbar(cax)
    ax.set_xticklabels(['']+list(corr.columns),rotation='vertical')
    ax.set_yticklabels(['']+list(corr.columns))
    tick_spacing = 1
    ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    ax.yaxis.set_label_coords(0, 0)
    ax.grid(False)
    plt.savefig(output_file_dir+'correlation.png')
    plt.show()
    
    print('Recursive feature elimination:')
    
    print('LDA + recall_macro:')
    rfecv = RFECV(estimator=lda, step=1, cv=StratifiedKFold(y2, 20), scoring='recall_macro')
    rfecv.fit(x_selected, y2)
    model = rfecv.estimator_
    x_selected_selected = x_selected[:,rfecv.support_]
    feature_name_selected = feature_name[rfecv.support_]
    y_pred = model.predict(x_selected_selected)
    print("Optimal number of features : %d" % rfecv.n_features_)
    print(feature_name_selected)
    print(model.coef_)
    print(classification_report(y2, y_pred))
    print('Leave-one-out cross validation accuracy:')
    print(str(np.mean(cross_val_score(lda,x_selected_selected,y2,cv=LeaveOneOut(len(y2)),scoring='accuracy'))))


    print('LDA + recall_weighted:')
    rfecv = RFECV(estimator=lda, step=1, cv=StratifiedKFold(y2, 20), scoring='recall_weighted')
    rfecv.fit(x_selected, y2)
    model = rfecv.estimator_
    x_selected_selected = x_selected[:,rfecv.support_]
    feature_name_selected = feature_name[rfecv.support_]
    y_pred = model.predict(x_selected_selected)
    print("Optimal number of features : %d" % rfecv.n_features_)
    print(feature_name_selected)
    print(model.coef_)
    print(classification_report(y2, y_pred))
    print('Leave-one-out cross validation accuracy:\n')
    print(str(np.mean(cross_val_score(lda,x_selected_selected,y2,cv=LeaveOneOut(len(y2)),scoring='accuracy'))))

    print('Logistic regression + recall_macro:')
    rfecv = RFECV(estimator=lr, step=1, cv=StratifiedKFold(y2, 20), scoring='recall_macro')
    rfecv.fit(x_selected, y2)
    model = rfecv.estimator_
    x_selected_selected = x_selected[:,rfecv.support_]
    feature_name_selected = feature_name[rfecv.support_]
    y_pred = model.predict(x_selected_selected)
    print("Optimal number of features : %d" % rfecv.n_features_)
    print(feature_name_selected)
    print(model.coef_)
    print(classification_report(y2, y_pred))
    lr.fit(x_selected_selected,y2)
    y_pred = lr.predict(x_selected_selected)
    train_sizes, train_scores, test_scores = learning_curve(lr,x_selected_selected,y2,scoring='accuracy', cv=LeaveOneOut(len(x_selected_selected)))
    print('Leave-one-out cross validation accuracy:')
    print(str(np.mean(cross_val_score(lr,x_selected_selected,y2,cv=LeaveOneOut(len(y2)),scoring='accuracy'))))
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.figure()
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")
    plt.title('Learning Curve')
    plt.xlabel('Training Sample Size')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(output_file_dir+'learning_curve_logistic_recall_macro.png')
    plt.show()

    print('Logistic regression + recall_weighted:')
    rfecv = RFECV(estimator=lr, step=1, cv=StratifiedKFold(y2, 20), scoring='recall_weighted')
    rfecv.fit(x_selected, y2)
    model = rfecv.estimator_
    x_selected_selected = x_selected[:,rfecv.support_]
    feature_name_selected = feature_name[rfecv.support_]
    y_pred = model.predict(x_selected_selected)
    print("Optimal number of features : %d" % rfecv.n_features_)
    print(feature_name_selected)
    print(model.coef_)
    print(classification_report(y2, y_pred))
    lr.fit(x_selected_selected,y2)
    y_pred = lr.predict(x_selected_selected)
    train_sizes, train_scores, test_scores = learning_curve(lr,x_selected_selected,y2,scoring='accuracy', cv=LeaveOneOut(len(x_selected_selected)))
    print('Leave-one-out cross validation accuracy:')
    print(str(np.mean(cross_val_score(lr,x_selected_selected,y2,cv=LeaveOneOut(len(y2)),scoring='accuracy'))))
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.figure()
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")
    plt.title('Learning Curve')
    plt.xlabel('Training Sample Size')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(output_file_dir+'learning_curve_logistic_recall_weighted.png')
    plt.show()

    print('SVC(linear) + recall_macro:')
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y2, 20), scoring='recall_macro')
    rfecv.fit(x_selected, y2)
    model = rfecv.estimator_
    x_selected_selected = x_selected[:,rfecv.support_]
    feature_name_selected = feature_name[rfecv.support_]
    y_pred = model.predict(x_selected_selected)
    print("Optimal number of features : %d" % rfecv.n_features_)
    print(feature_name_selected)
    print(model.coef_)
    print(classification_report(y2, y_pred))
    svc.fit(x_selected_selected,y2)
    y_pred = svc.predict(x_selected_selected)
    train_sizes, train_scores, test_scores = learning_curve(svc,x_selected_selected,y2,scoring='accuracy', cv=LeaveOneOut(len(x_selected_selected)))
    print('Leave-one-out cross validation accuracy:\n')
    print(str(np.mean(cross_val_score(svc,x_selected_selected,y2,cv=LeaveOneOut(len(y2)),scoring='accuracy'))))
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.figure()
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")
    plt.title('Learning Curve')
    plt.xlabel('Training Sample Size')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(output_file_dir+'learning_curve_svc_recall_macro.png')
    plt.show()
    
    print('SVC(linear) + recall_weighted:\n')
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y2, 20), scoring='recall_weighted')
    rfecv.fit(x_selected, y2)
    model = rfecv.estimator_
    x_selected_selected = x_selected[:,rfecv.support_]
    feature_name_selected = feature_name[rfecv.support_]
    y_pred = model.predict(x_selected_selected)
    print("Optimal number of features : %d" % rfecv.n_features_)
    print(feature_name_selected)
    print(model.coef_)
    print(classification_report(y2, y_pred))
    svc.fit(x_selected_selected,y2)
    y_pred = svc.predict(x_selected_selected)
    train_sizes, train_scores, test_scores = learning_curve(svc,x_selected_selected,y2,scoring='accuracy', cv=LeaveOneOut(len(x_selected_selected)))
    print('Leave-one-out cross validation accuracy:\n')
    print(str(np.mean(cross_val_score(svc,x_selected_selected,y2,cv=LeaveOneOut(len(y2)),scoring='accuracy'))))
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.figure()
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation score")
    plt.title('Learning Curve')
    plt.xlabel('Training Sample Size')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(output_file_dir+'learning_curve_svc_recall_weighted.png')
    plt.show()


Use PaDEL 1D+2D descriptors

In [ ]:
model_generation('../data/features.csv','../data/logBB.txt','../data/features/')

Use PubChem fingerprint

In [ ]:
model_generation('../data/features2.csv','../data/logBB.txt','../data/features2/')

Use PaDEL 1D+2D descriptors + PubChem fingerprint

In [ ]:
model_generation('../data/features3.csv','../data/logBB.txt','../data/features3/')